In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Pré-Processamento
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Classificadores
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Avaliação
from sklearn.metrics import accuracy_score, precision_score, recall_score,  roc_curve, auc
from sklearn.model_selection import KFold, cross_validate

import warnings
warnings.filterwarnings('ignore')

C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df=pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,caa,output
0,63,1,3,145,233,1,0,150,0,0,1
1,37,1,2,130,250,0,1,187,0,0,1
2,41,0,1,130,204,0,0,172,0,0,1
3,56,1,1,120,236,0,1,178,0,0,1
4,57,0,0,120,354,0,1,163,1,0,1


In [3]:
df.shape

(303, 11)

In [4]:
# Separando as vaiáveis em categóricas e numéricas 
cat_cols = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'caa']
num_cols = ['age','trtbps', 'chol', 'thalachh']

### EDA (Exploratory Data Analysis) 
Este passo foi realizado no notebook sobre análise de dados

Neste notebook vamos desenvolver um modelo de calssificação binária

<div>
<center><img src="imgs/class_problems.png" width="600"/></center>
</div>

Lembrando:
- Binária: 2 classes na variável resposta
- Multilabel: N classes mutuamente exclusivos. (um gato não pode ser cachorro e gato ao mesmo tempo)
- Multiclasse: N classes, sendo que uma resposta pode conter mais de uma classe. (Um filme pode ser ao mesmo tempo romance e aventura)

### Separando os conjuntos para a modelagem

- Base de Treino: Conjunto de características e conjunto de resposta. São os dados com os quais o modelo vai aprender
- Base de Teste: Conjunto de dados em que vamos testar se o modelo aprendeu bem

In [5]:
X = df.drop('output', axis = 1)
y = df['output']

In [6]:
# Sparamos 80% para aprendizado e 20% para treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
print(X_train.shape, X_test.shape)

(242, 10) (61, 10)


Quando a característica do nosso dado possui uma dependência/relação temporal, não podemos fazer essa seleção puramente aleatória. Devemos considerar a relação temporal

### Pré-processamento

**Normalização :** Normalizamos os dados numéricos para "não correr o risco" que o modelo possa dar um peso maior ou menor para uma variável apenas por sua escala. 

In [7]:
scaler = StandardScaler()
X_train.loc[:,num_cols] = scaler.fit_transform(X_train.loc[:, num_cols])
X_test.loc[:,num_cols] = scaler.transform(X_test.loc[:, num_cols])

### Processo de Modelagem

1. Escolha o seu classificador e busque na documentação como usá-lo
2. A função *fit* é responsável pelo aprendizado, e por isso ela é aplicada na base de treino.
3. A função *predict* irá utilizar o aprendizado adquirido anteriormente para classificar um novo conjunto de dados. Por isso é aplicado em um conjunto de teste.

**LEMBRE-SE! TIRAR O RESULTADO DO MODELO APENAS COM BASE NOS RESULTADOS DE TREINAMENTO, É COMO TESTAR SEUS CONHECIMENTOS EM UMA PROVA NA QUAL VOCÊ TEVE ACESSO AS PERGUNTAR QUE CAIRIAM NA PROVA. NESSE CASO VOCÊ DECOROU APENAS A RESPOSTA PARA AQUELAS PERGUNTAS E NÃO A MATÉRIA. O MODELO DEVE SER SUBMETIDO A UMA PROVA NA QUAL NUNCA VIU AS QUESTÕES**

Sobre os parâmetros do modelo, quais usar? Você pode testar alguns deles. Sobre como programar estes testes, consulte os links a seguir :
- https://medium.com/fintechexplained/what-is-grid-search-c01fe886ef0a
- https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search

**Enconding das variáveis Categóricas :** Nesta bases de dados, as variáveis categóricas já estão codificadas como números. Apenas como exemplo, ao final do notebook está um exemplo em como usar LabelEncoder

**Regressão Logística**

<div>
<center><img src="imgs/logistic_reg.png" width="600"/></center>
</div>

Na regressão logística, vamos tentar traçar uma curva sigmóide (essa aí que parece um S) no espaço n-dimensional onde estão plotados todos as nossas características. O melhor ajuste para essa curva define o nosso modelo.

Na imagem acima estamos tentando prever se um grupo de pessoas é obeso ou não com base no peso. O melhor ajuste da curva vai nos mostrar que a partir de um determinado peso, a pessoa será considerada obesa. Claro que este é um exemplo bem simples, considera uma única variável, o peso. Precisamos de uma técnica de ML pois não seríamos capazes de analisar muitas variáveis ao mesmo tempo.

In [8]:
log_model = LogisticRegression(solver='liblinear') # escolha da técnica
log_model.fit(X_train, y_train) # aprendizado

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
# Predição na base de teste
log_model_pred = log_model.predict(X_test)

Note a diferença entre eles

In [10]:
log_model_pred

array([0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0], dtype=int64)

Note também que o resultado da predição já foi a própria categoria. No entanto, podemos utilizar uma outra função predict_proba* para que se retorne a probabilidade de cada categoria

In [11]:
log_model.predict_proba(X_test)

array([[0.94372637, 0.05627363],
       [0.46665346, 0.53334654],
       [0.04759857, 0.95240143],
       [0.98845969, 0.01154031],
       [0.91787965, 0.08212035],
       [0.83860656, 0.16139344],
       [0.97178807, 0.02821193],
       [0.40226293, 0.59773707],
       [0.96733914, 0.03266086],
       [0.99344309, 0.00655691],
       [0.37084425, 0.62915575],
       [0.17833845, 0.82166155],
       [0.98117945, 0.01882055],
       [0.92787517, 0.07212483],
       [0.00525711, 0.99474289],
       [0.91341835, 0.08658165],
       [0.6798797 , 0.3201203 ],
       [0.41168749, 0.58831251],
       [0.23840403, 0.76159597],
       [0.98112632, 0.01887368],
       [0.03140101, 0.96859899],
       [0.9813041 , 0.0186959 ],
       [0.06873311, 0.93126689],
       [0.26375891, 0.73624109],
       [0.21857823, 0.78142177],
       [0.94141996, 0.05858004],
       [0.97169418, 0.02830582],
       [0.94233052, 0.05766948],
       [0.02658244, 0.97341756],
       [0.10942144, 0.89057856],
       [0.

Por default o python escolhe a categoria quando probabilidade é >50%. Com o *predict_proba* podemos ter mais controle nesta decisão, se isto for relevante para o problema. Neste caso vamos usar o default mesmo. 

**Árvores de Decisão**

<div>
<center><img src="imgs/dec_tree.png" width="600"/></center>
</div>

A árvore de decisão é um modelo bem intuitivo. Ele quebra o problema variável por variável (no qual um cálculo por trás diz qual é essa ordem de quebra da variável), até chegar na classificação final.

Neste repositório existe um notebook específico para árvore de decisão. Case deseje saber mais um pouco pode consultar.

In [12]:
decision_model = DecisionTreeClassifier(max_depth=4) # escolha da técnica
decision_model.fit(X_train, y_train) # aprendizado 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
# Predição na base de teste
decision_model_pred = decision_model.predict(X_test)

**Random Forest**

<div>
<center><img src="imgs/rf.png" width="600"/></center>
</div>

O Random Forest é um algoritimo do tipo *Ensemble*, ou seja, ele é uma combinação de outros algoritmos, neste caso de árvores de decisão. Ele executa múltiplas árvores de decisão, cada uma com um subconjunto de variáveis. Para o resultado final da classificação é feita uma "votação" entre todas as árvores e é decidida aquela que possui a maior quantidade de votos.

In [14]:
rf = RandomForestClassifier(criterion='entropy', max_depth=2) # escolha da técnica
rf.fit(X_train, y_train) # aprendizado 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
# Predição na base de teste
rf_pred = rf.predict(X_test)

### Métodos de avaliação

Agora que já treinamos o nosso modelo e fizemos uma predição com uma base de teste, vamos avaliar estes resultados

**Matriz de confusão**

| - | SIM (predito) | NÃO (predito) |
| --------  | ------------------- | --------------------- |
| **SIM (real)**  | Verdadeiro Positivo (VP) | False Negativo (FN)| 
| **NÃO (real)**  | Falso Positivo (FP) | Verdadeiro Negativo (VN) | 

- Verdadeiros Positivos: classificação correta da classe Positivo
- Falsos Negativos: erro em que o modelo previu a classe Negativo quando o valor real era classe Positivo
- Falsos Positivos: erro em que o modelo previu a classe Positivo quando o valor real era classe Negativo
- Verdadeiros Negativos: classificação correta da classe Negativo

**Métricas de Avaliação**
- Acurácia: indica uma performance geral do modelo. Dentre todas as classificações, quantas o modelo classificou corretamente (VP + VN)/(VP + VN + FP + FN)
- Precisão: dentre todas as classificações de classe Positivo que o modelo fez, quantas estão corretas VP/(VP+FP)
- Recall/Revocação/Sensibilidade: dentre todas as situações de classe Positivo como valor esperado, quantas estão corretas VP/(VP+FN)

**Regressão Logística**

In [16]:
print("Avaliação acurácia:", accuracy_score(y_test, log_model_pred))
print("Avaliação precisão:",precision_score(y_test, log_model_pred))
print("Avaliação recall:",recall_score(y_test, log_model_pred))

Avaliação acurácia: 0.8032786885245902
Avaliação precisão: 0.8148148148148148
Avaliação recall: 0.7586206896551724


**Árvore de Decisão**

In [17]:
print(accuracy_score(y_test, decision_model.predict(X_test)))
print(precision_score(y_test, decision_model.predict(X_test)))
print(recall_score(y_test, decision_model.predict(X_test)))

0.819672131147541
0.8214285714285714
0.7931034482758621


**Random Forest**

In [18]:
print(accuracy_score(y_test, rf.predict(X_test)))
print(precision_score(y_test, rf.predict(X_test)))
print(recall_score(y_test, rf.predict(X_test)))

0.7540983606557377
0.75
0.7241379310344828


### Validação Cruzada

<div>
<center><img src="imgs/kfold.jpg" width="600"/></center>
</div>

A validação cruzada é uma outra forma de avaliar o modelo. Desta vez, a base de dados é fatiada em seções, e não mais uma seleção aleatória como feito anteriormente. Desta forma é possível entender se o seu conjunto de dados possui comportamentos distintos em porções distintas. Dessa forma caso o algoritmo tenha "visto" uma porção específica, com um comportamento específico ele acaba não sendo um modelo que for capaz de generalizar. Ou seja, ele só será capaz de predizer de forma correta de vir novamente aquele comportamente específico.

Esta metodologia é ideal para encontrar sazonalidade.

**! O algoritmo KFold possui um parâmetro específico de shuffle, ou seja, você pode embaralhar os dados antes de fazer as divisões ou manter a sequência inicial. Vejamos as diferenças**

In [19]:
# Sem Shuffle
k = 4
kf = KFold(n_splits=k, shuffle=False)
model = LogisticRegression(solver='liblinear')

acc_score = []
 
for train_index , test_index in kf.split(X):
    X_train_cv , X_test_cv = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train_cv , y_test_cv = y[train_index] , y[test_index]
     
    model.fit(X_train_cv,y_train_cv)
    pred_values = model.predict(X_test_cv)
     
    acc = accuracy_score(pred_values , y_test_cv)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.8026315789473685, 0.6710526315789473, 0.6447368421052632, 0.5733333333333334]
Avg accuracy : 0.6729385964912281


In [20]:
# Sem Shuffle
k = 4
kf = KFold(n_splits=k, shuffle=True)
model = LogisticRegression(solver='liblinear')

acc_score = []
 
for train_index , test_index in kf.split(X):
    X_train_cv , X_test_cv = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train_cv , y_test_cv = y[train_index] , y[test_index]
     
    model.fit(X_train_cv,y_train_cv)
    pred_values = model.predict(X_test_cv)
     
    acc = accuracy_score(pred_values , y_test_cv)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.7368421052631579, 0.8026315789473685, 0.8552631578947368, 0.8133333333333334]
Avg accuracy : 0.8020175438596492


Note a mundança na performace. Isso ocorre pois quanto mais bem distribuídos os dados estão, mais o algoritmo é capaz de generalizar. Claro que o fato da base ser pequena isto também é agravado.

### Anexo: Econding de variáveis categóricas

In [21]:
# Considere como exemplo a variável sex com duas categórias Fem e Masc
exemplo = np.where(X_train['sex'] == 1, "Fem", "Masc")
exemplo

array(['Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Masc',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Masc', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc',
       'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc',
       'Fem', 'Masc', 'Masc', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Masc', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem',
       'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Masc', 'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', '

In [22]:
# O Label encoder irá pegar todas as classes e irá transformar em "números"
le = preprocessing.LabelEncoder()
le.fit(exemplo)
le.classes_ # quantas classes únicas foram encontradas

array(['Fem', 'Masc'], dtype='<U4')

In [23]:
# Aplicação do encoding
exemplo_cod = le.transform(exemplo)
exemplo_cod

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
      dtype=int64)

In [24]:
# Processo reverso
le.inverse_transform(exemplo_cod)

array(['Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Masc',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Masc', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc',
       'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc',
       'Fem', 'Masc', 'Masc', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Masc', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem',
       'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Masc', 'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', '